In [59]:
# LIBRARIES
import pandas as pd
import json

In [60]:
# DATA ACQUISITION
data = pd.read_csv('https://raw.githubusercontent.com/juvenalfonseca/python/master/datasets/campeonato-brasileiro-pontos-corridos-2003-2020-jogos.csv', delimiter = ';')
#data_p = pd.read_csv('https://raw.githubusercontent.com/juvenalfonseca/python/master/datasets/campeonato-brasileiro-pontos-corridos-2003-2020-periodo.csv', delimiter = ';')

In [61]:
# DATA PREPARING
data['Ano'] = data['Data'].str.slice(start=6,stop=10)
data['Ano'] = data['Ano'].str.replace('2021','2020')

data = data[['Rodada','Mandante','Visitante','Vencedor','Mandante Placar','Visitante Placar','Ano']].sort_values(by=['Rodada'])

In [63]:
pontos = []
for i in data.index:
    if data['Vencedor'][i] == '-':
        pontos.append([data['Ano'][i], data['Rodada'][i], data['Mandante'][i], 1])
        pontos.append([data['Ano'][i], data['Rodada'][i], data['Visitante'][i], 1])

    elif data['Vencedor'][i] == data['Mandante'][i]:
        pontos.append([data['Ano'][i], data['Rodada'][i], data['Mandante'][i], 3])
        pontos.append([data['Ano'][i], data['Rodada'][i], data['Visitante'][i], 0])

    elif data['Vencedor'][i] == data['Visitante'][i]:
        pontos.append([data['Ano'][i], data['Rodada'][i], data['Mandante'][i], 0])
        pontos.append([data['Ano'][i], data['Rodada'][i], data['Visitante'][i], 3])

In [64]:
df_p = pd.DataFrame(pontos, columns=['Ano','Rodada','Time','Pontos'])

In [65]:
ano = df_p['Ano'].unique().tolist()
for a in ano:
    subset_1 = df_p.loc[df_p['Ano'] == a]
    times = subset_1['Time'].unique().tolist()
    for b in times:
        i = max(subset_1['Rodada'])
        while i > 0:
            subset = subset_1.loc[(subset_1['Rodada'] <= i) & (subset_1['Time'] == b)]
            df_p.loc[(df_p['Ano'] == a) & (df_p['Rodada'] == i) & (df_p['Time'] == b), 'PontosAcum'] = subset['Pontos'].sum()
            i = i-1

In [66]:
df_p_2020 = df_p.loc[df_p['Ano'] == '2020']

In [67]:
df_f = pd.concat([pd.DataFrame([df_p_2020.columns.tolist()], columns=df_p_2020.columns.tolist()), df_p_2020])

In [68]:
# DATA EXPORT
result = df_f.to_json(orient='values')
parsed = json.loads(result)
with open('sbrasileirao.json', 'w', encoding='utf-8') as f:
    json.dump(parsed, f, ensure_ascii=False)